# plansquad template generation (Markdown)

Build a planning template as structured Markdown. This takes the form of a graph where we generate markdown in the first node, then pass it to a conversion and validation tool. (This might be better done by fine-tuning a model to do the conversion in a single step.) 


In [ ]:
%%capture --no-stderr
%pip install -U langgraph langsmith
%pip install -U tavily-python
%pip install -U langchain_community
%pip install -U langchain-groq


## Set up the environment

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "PlanSquad Intake Test"
# Make sure we're using the decrypted secrets
assert(os.environ["LANGSMITH_API_KEY"].startswith("lsv2_"))

## Load a chat engine

In [ ]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant")

## Build a prompt template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# From https://smith.langchain.com/hub/mcdiddy/knowledgebasejsoniterator

prompt = ChatPromptTemplate.from_messages([
  ("system", '''
  You are a helpful, skilled planner who assists people in planning their
  most important events. But before you create a detailed plan, there are
  things you need to know.

  You are going to create an outline with the questions it takes to make
  a detailed plan with them. I want you to create it in the following format: 
  {format_details}
  
  For example, if they are planning a birthday party, your outline might 
  look like this:

```{format}
{format_sample}
```

Here are the instructions for you to follow:
1. Give a friendly greeting and ask what they would like to plan.
2. Do NOT ask them questions or make suggestions.
3. Offer them an outline like the one above. It should be as complete as you can make it.
4. Your outline will be in Markdown format. Avoid any extraneous text that isn't in {format}.


'''),
  ("human", "{question}"),
])


## Generate template

### Markdown sample

In [ ]:
format = "markdown"
format_details = '''
# Short name such as "Birthday Party Planner"

A short description of what this plan is for.

## First section name

1. Question
2. Question
3. Question

## Second section name

1. Question
2. Question
3. Question

'''

format_sample = '''
# Birthday Party Plan
# Birthday Party planner

This will help you plan the perfect birthday party! Please fill in any information
you can. 

## The basics

1. Who is this party for?
2. When do you want this to happen?
3. Who will you invite?

## The venue

1. What is the venue?
2. What is its availability?
3. How much does it cost?
'''

In [ ]:
messages  = prompt.invoke(
    {"question": '''
I want to create training for a group of salespeople.
''', 
    "format": format, 
    "format_details": format_details,
    "format_sample": format_sample
}).messages

model.invoke(messages)